## CNN Classifier

This notebook contains the code for the CNN classifier, trained with the full training dataset (i.e. no data reduction has been performed). Note that 5-fold cross-validation has been used to optimize the hyperparameters of the CNN. 

In [1]:
import tensorflow as tf # tensorflow library
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold

In [2]:
# Loading in 2000 raw image files, which are encoded as row-vectors of size 240.
mfeat_pix = np.loadtxt("mfeat-pix.txt")

# Splitting the data into a training and testing dataset.
trainset = np.concatenate((mfeat_pix[0:100], mfeat_pix[200:300], 
                               mfeat_pix[400:500], mfeat_pix[600:700],
                               mfeat_pix[800:900], mfeat_pix[1000:1100],
                               mfeat_pix[1200:1300], mfeat_pix[1400:1500],
                               mfeat_pix[1600:1700], mfeat_pix[1800:1900]))

# Converting and re-shaping the train vectors to 16 by 15 NumPy arrays containing elements of type 'float32'.
x_train = np.float32(np.array([np.reshape(trainset[i], (16, 15)) for i in range(len(trainset))]))
b = np.array([1]*100)
y_train = np.concatenate((b*0, b, b*2, b*3, b*4, b*5, b*6, b*7, b*8, b*9))

testset = np.concatenate((mfeat_pix[100:200], mfeat_pix[300:400], 
                               mfeat_pix[500:600], mfeat_pix[700:800],
                               mfeat_pix[900:1000], mfeat_pix[1100:1200],
                               mfeat_pix[1300:1400], mfeat_pix[1500:1600],
                               mfeat_pix[1700:1800], mfeat_pix[1900:2000]))

# Converting and re-shaping the test vectors to 16 by 15 NumPy arrays containing elements of type 'float32'.
x_test = np.float32(np.array([np.reshape(testset[i], (16, 15)) for i in range(len(testset))]))
y_test = y_train


In [3]:
# Normalizing the train dataset.
x_train = tf.keras.utils.normalize(x_train, axis=1)
# Normalizing the test dataset.
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [4]:
def CNN(filters=50, kernel_size=3, drop_out=0.4):
    """This function represents our Convolutional Neural Network (CNN)."""
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((16, 15, 1), input_shape=x_train.shape[1:]),
    tf.keras.layers.Conv2D(filters, kernel_size, padding='same', strides=(2, 2), 
                           activation='relu', input_shape=x_train.shape[1:]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(drop_out),
    
    tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(drop_out),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid')
    ])
    
    return model 

## Test run with CNN

In [5]:
# Compile the model
model = CNN(filters=50, kernel_size=3, drop_out=0.4)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 16, 15, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 50)          500       
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 50)         200       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 50)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 4, 4, 50)          0         
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 50)          2

In [6]:
# Fitting the model to the training data.
no_epochs = 50
model.fit(x_train, y_train, epochs=no_epochs) 

Epoch 1/50
32/32 [==============================] - 1s 16ms/step - loss: 2.3494 - accuracy: 0.2840
Epoch 2/50
32/32 [==============================] - 1s 26ms/step - loss: 1.2292 - accuracy: 0.5910
Epoch 3/50
32/32 [==============================] - 1s 26ms/step - loss: 0.8239 - accuracy: 0.7300
Epoch 4/50
32/32 [==============================] - 1s 31ms/step - loss: 0.5921 - accuracy: 0.7980
Epoch 5/50
32/32 [==============================] - 1s 26ms/step - loss: 0.4228 - accuracy: 0.8730
Epoch 6/50
32/32 [==============================] - 1s 37ms/step - loss: 0.3568 - accuracy: 0.8810
Epoch 7/50
32/32 [==============================] - 1s 25ms/step - loss: 0.2598 - accuracy: 0.9210
Epoch 8/50
32/32 [==============================] - 1s 18ms/step - loss: 0.2758 - accuracy: 0.9150
Epoch 9/50
32/32 [==============================] - 1s 18ms/step - loss: 0.2060 - accuracy: 0.9430
Epoch 10/50
32/32 [==============================] - 1s 20ms/step - loss: 0.1962 - accuracy: 0.9380
Epoch 11/

In [7]:
# Evaluating the model performance on the test dataset.
test_loss, test_acc = model.evaluate(x_test, y_test)
print('\nTest accuracy: {:.2f}%'.format(test_acc*100))

32/32 [==============================] - 0s 1ms/step - loss: 0.0944 - accuracy: 0.9810

Test accuracy: 98.10%


## CNN with 5-Fold Cross-Validation

In [9]:
def loop(no_epochs, filters, batch_size, drop_out, variable, no_folds=5, inputs=x_train, targets=y_train):
    """This function contains the for loop which we use to perform 5-fold cross-validation on the number of epochs,
    the number of filters, the batch size and the drop-out constant in the drop-out layers using the training data."""
    
    acc_per_fold = []   # for storing accuracy per fold
    loss_per_fold = []   # for storing loss per fold

    # Initializing the k-fold cross-validation.
    no_folds = 5   # number of folds
    kfold = KFold(n_splits=no_folds, shuffle=True)

    # This for loop performs k-fold cross-validation on the training data. 
    fold_no = 1
    for train, test in kfold.split(inputs, targets):
        
        # Initializing our CNN.
        model = CNN(filters=filters, kernel_size=3, drop_out=drop_out)

        # Compiling the model.
        model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

        # Marking the beginning of each fold with a print statement.
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fitting the model to the training data.
        history = model.fit(inputs[train], targets[train], batch_size=batch_size, epochs=no_epochs)

        # Model evaluation.
        val_loss, val_acc = model.evaluate(inputs[test], targets[test], batch_size=batch_size)
        acc_per_fold.append(val_acc*100)
        loss_per_fold.append(val_loss)

        # Increasing fold number.
        fold_no += 1

    accs['{}'.format(variable)] = {'mean': np.mean(acc_per_fold), 'std': np.std(acc_per_fold)}
    losses['{}'.format(variable)] = np.mean(loss_per_fold)
    

In [10]:
"""Performing 5-fold cross-validation on the batch size."""

no_epochs = 50
no_filters = 50
batch_sizes = np.arange(16, 80, 16)
drop_out = 0.4

accs = {}
losses = {}

for b in batch_sizes:
    loop(no_epochs=no_epochs, filters=no_filters, batch_size=b, drop_out=drop_out,
         variable='batch_size = {}'.format(b), no_folds=5)
                

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
50/50 [==============================] - 1s 19ms/step - loss: 2.0185 - accuracy: 0.3613
Epoch 2/50
50/50 [==============================] - 1s 17ms/step - loss: 1.0028 - accuracy: 0.6600
Epoch 3/50
50/50 [==============================] - 1s 14ms/step - loss: 0.6407 - accuracy: 0.7850
Epoch 4/50
50/50 [==============================] - 1s 21ms/step - loss: 0.5190 - accuracy: 0.8250
Epoch 5/50
50/50 [==============================] - 1s 22ms/step - loss: 0.3981 - accuracy: 0.8675
Epoch 6/50
50/50 [==============================] - 1s 23ms/step - loss: 0.3347 - accuracy: 0.8850
Epoch 7/50
50/50 [==============================] - 1s 19ms/step - loss: 0.2569 - accuracy: 0.9162
Epoch 8/50
50/50 [==============================] - 1s 21ms/step - loss: 0.2453 - accuracy: 0.9100
Epoch 9/50
50/50 [==============================] - 1s 26ms/step - loss: 0.2357 - accuracy: 0.9287
Epoch 10/50


50/50 [==============================] - 1s 18ms/step - loss: 0.2056 - accuracy: 0.9400
Epoch 9/50
50/50 [==============================] - 1s 23ms/step - loss: 0.2072 - accuracy: 0.9187
Epoch 10/50
50/50 [==============================] - 1s 18ms/step - loss: 0.1610 - accuracy: 0.9588
Epoch 11/50
50/50 [==============================] - 1s 25ms/step - loss: 0.1678 - accuracy: 0.9425
Epoch 12/50
50/50 [==============================] - 1s 20ms/step - loss: 0.1398 - accuracy: 0.9550
Epoch 13/50
50/50 [==============================] - 1s 15ms/step - loss: 0.1131 - accuracy: 0.9638
Epoch 14/50
50/50 [==============================] - 1s 19ms/step - loss: 0.1372 - accuracy: 0.9525
Epoch 15/50
50/50 [==============================] - 1s 19ms/step - loss: 0.0955 - accuracy: 0.9712
Epoch 16/50
50/50 [==============================] - 1s 29ms/step - loss: 0.0805 - accuracy: 0.9688
Epoch 17/50
50/50 [==============================] - 1s 22ms/step - loss: 0.0882 - accuracy: 0.9712
Epoch 18/50
5

25/25 [==============================] - 0s 15ms/step - loss: 0.1121 - accuracy: 0.9588
Epoch 17/50
25/25 [==============================] - 1s 20ms/step - loss: 0.1206 - accuracy: 0.9588
Epoch 18/50
25/25 [==============================] - 0s 20ms/step - loss: 0.1007 - accuracy: 0.9663
Epoch 19/50
25/25 [==============================] - 1s 22ms/step - loss: 0.0930 - accuracy: 0.9675
Epoch 20/50
25/25 [==============================] - 1s 26ms/step - loss: 0.1057 - accuracy: 0.9688
Epoch 21/50
25/25 [==============================] - 1s 23ms/step - loss: 0.0984 - accuracy: 0.9712
Epoch 22/50
25/25 [==============================] - 1s 19ms/step - loss: 0.0737 - accuracy: 0.9787
Epoch 23/50
25/25 [==============================] - 1s 23ms/step - loss: 0.0723 - accuracy: 0.9762
Epoch 24/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0730 - accuracy: 0.9775
Epoch 25/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0399 - accuracy: 0.9887
Epoch 26/50


25/25 [==============================] - 1s 33ms/step - loss: 0.0546 - accuracy: 0.9787
Epoch 25/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0596 - accuracy: 0.9725
Epoch 26/50
25/25 [==============================] - 1s 20ms/step - loss: 0.0526 - accuracy: 0.9812
Epoch 27/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0511 - accuracy: 0.9862
Epoch 28/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0663 - accuracy: 0.9762
Epoch 29/50
25/25 [==============================] - 0s 17ms/step - loss: 0.0763 - accuracy: 0.9725
Epoch 30/50
25/25 [==============================] - 1s 21ms/step - loss: 0.0595 - accuracy: 0.9787
Epoch 31/50
25/25 [==============================] - 1s 21ms/step - loss: 0.0554 - accuracy: 0.9800
Epoch 32/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0649 - accuracy: 0.9812
Epoch 33/50
25/25 [==============================] - 1s 30ms/step - loss: 0.0551 - accuracy: 0.9775
Epoch 34/50


17/17 [==============================] - 0s 21ms/step - loss: 0.0587 - accuracy: 0.9825
Epoch 33/50
17/17 [==============================] - 0s 15ms/step - loss: 0.0643 - accuracy: 0.9812
Epoch 34/50
17/17 [==============================] - 1s 38ms/step - loss: 0.0477 - accuracy: 0.9875
Epoch 35/50
17/17 [==============================] - 1s 29ms/step - loss: 0.0540 - accuracy: 0.9837
Epoch 36/50
17/17 [==============================] - 0s 19ms/step - loss: 0.0548 - accuracy: 0.9812
Epoch 37/50
17/17 [==============================] - 0s 19ms/step - loss: 0.0388 - accuracy: 0.9875
Epoch 38/50
17/17 [==============================] - 0s 15ms/step - loss: 0.0352 - accuracy: 0.9912
Epoch 39/50
17/17 [==============================] - 0s 19ms/step - loss: 0.0399 - accuracy: 0.9875
Epoch 40/50
17/17 [==============================] - 0s 25ms/step - loss: 0.0357 - accuracy: 0.9912
Epoch 41/50
17/17 [==============================] - 0s 21ms/step - loss: 0.0534 - accuracy: 0.9825
Epoch 42/50


13/13 [==============================] - 0s 15ms/step - loss: 0.0638 - accuracy: 0.9737
Epoch 41/50
13/13 [==============================] - 0s 21ms/step - loss: 0.0430 - accuracy: 0.9887
Epoch 42/50
13/13 [==============================] - 0s 29ms/step - loss: 0.0466 - accuracy: 0.9825
Epoch 43/50
13/13 [==============================] - 0s 24ms/step - loss: 0.0527 - accuracy: 0.9775
Epoch 44/50
13/13 [==============================] - 0s 27ms/step - loss: 0.0504 - accuracy: 0.9812
Epoch 45/50
13/13 [==============================] - 0s 19ms/step - loss: 0.0469 - accuracy: 0.9875
Epoch 46/50
13/13 [==============================] - 0s 17ms/step - loss: 0.0411 - accuracy: 0.9862
Epoch 47/50
13/13 [==============================] - 0s 20ms/step - loss: 0.0392 - accuracy: 0.9862
Epoch 48/50
13/13 [==============================] - 0s 22ms/step - loss: 0.0532 - accuracy: 0.9800
Epoch 49/50
13/13 [==============================] - 0s 16ms/step - loss: 0.0438 - accuracy: 0.9850
Epoch 50/50


13/13 [==============================] - 0s 25ms/step - loss: 0.0335 - accuracy: 0.9925
Epoch 49/50
13/13 [==============================] - 0s 22ms/step - loss: 0.0355 - accuracy: 0.9862
Epoch 50/50
4/4 [==============================] - 0s 2ms/step - loss: 0.0841 - accuracy: 0.9800
------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/50
13/13 [==============================] - 0s 5ms/step - loss: 3.1668 - accuracy: 0.1813
Epoch 2/50
13/13 [==============================] - 0s 32ms/step - loss: 1.7165 - accuracy: 0.4013
Epoch 3/50
13/13 [==============================] - 0s 37ms/step - loss: 1.2175 - accuracy: 0.5763
Epoch 4/50
13/13 [==============================] - 1s 39ms/step - loss: 1.0143 - accuracy: 0.6413
Epoch 5/50
13/13 [==============================] - 0s 38ms/step - loss: 0.8070 - accuracy: 0.7300
Epoch 6/50
13/13 [==============================] - 0s 19ms/step - loss: 0.6309 - accuracy: 0.7900
Epoch 7/50
13/13 [=======

In [11]:
print("Dictionary containing mean accuracies of each round of 5-fold cross-validation:")
print(accs)
print()

max_acc = max(i['mean'] for i in accs.values())
mean_accs = {key: max(val.values()) for key, val in accs.items()}
batch_opt = max(mean_accs, key=mean_accs.get)
print("The highest validation accuracy of {:.2f}% is achieved with {}.".format(max_acc, batch_opt))

Dictionary containing mean accuracies of each round of 5-fold cross-validation:
{'batch_size = 16': {'mean': 97.30000019073486, 'std': 1.3638195787239105}, 'batch_size = 32': {'mean': 97.90000081062317, 'std': 1.496663837232324}, 'batch_size = 48': {'mean': 97.2000002861023, 'std': 0.8717815554213909}, 'batch_size = 64': {'mean': 97.10000038146973, 'std': 1.3564670758202029}}

The highest validation accuracy of 97.90% is achieved with batch_size = 32.


In [12]:
"""Performing 5-fold cross-validation on the drop_out parameter."""

no_epochs = 50
no_filters = 50
batch_size = int(batch_opt[-2:])
drop_outs = [0.2, 0.3, 0.4, 0.5]

accs = {}
losses = {}

for d in drop_outs:
    loop(no_epochs=no_epochs, filters=no_filters, batch_size=batch_size, drop_out=d,
         variable='drop_out = {}'.format(d), no_folds=5)
                

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
25/25 [==============================] - 1s 23ms/step - loss: 1.8442 - accuracy: 0.4338
Epoch 2/50
25/25 [==============================] - 1s 22ms/step - loss: 0.6819 - accuracy: 0.7987
Epoch 3/50
25/25 [==============================] - 0s 19ms/step - loss: 0.4067 - accuracy: 0.8850
Epoch 4/50
25/25 [==============================] - 1s 21ms/step - loss: 0.2697 - accuracy: 0.9150
Epoch 5/50
25/25 [==============================] - 0s 18ms/step - loss: 0.1789 - accuracy: 0.9538
Epoch 6/50
25/25 [==============================] - 1s 24ms/step - loss: 0.1185 - accuracy: 0.9712
Epoch 7/50
25/25 [==============================] - 0s 17ms/step - loss: 0.1105 - accuracy: 0.9625
Epoch 8/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0967 - accuracy: 0.9737
Epoch 9/50
25/25 [==============================] - 1s 35ms/step - loss: 0.0618 - accuracy: 0.9800
Epoch 10/50


25/25 [==============================] - 1s 24ms/step - loss: 0.0805 - accuracy: 0.9775
Epoch 9/50
25/25 [==============================] - 0s 17ms/step - loss: 0.0596 - accuracy: 0.9862
Epoch 10/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0585 - accuracy: 0.9862
Epoch 11/50
25/25 [==============================] - 1s 20ms/step - loss: 0.0581 - accuracy: 0.9850
Epoch 12/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0408 - accuracy: 0.9900
Epoch 13/50
25/25 [==============================] - 0s 20ms/step - loss: 0.0322 - accuracy: 0.9962
Epoch 14/50
25/25 [==============================] - 0s 17ms/step - loss: 0.0261 - accuracy: 0.9962
Epoch 15/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0248 - accuracy: 0.9962
Epoch 16/50
25/25 [==============================] - 0s 21ms/step - loss: 0.0294 - accuracy: 0.9912
Epoch 17/50
25/25 [==============================] - 1s 27ms/step - loss: 0.0220 - accuracy: 0.9925
Epoch 18/50
2

25/25 [==============================] - 1s 22ms/step - loss: 0.0470 - accuracy: 0.9837
Epoch 17/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0436 - accuracy: 0.9862
Epoch 18/50
25/25 [==============================] - 0s 18ms/step - loss: 0.0344 - accuracy: 0.9925
Epoch 19/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0332 - accuracy: 0.9875
Epoch 20/50
25/25 [==============================] - 0s 12ms/step - loss: 0.0354 - accuracy: 0.9862
Epoch 21/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0298 - accuracy: 0.9912
Epoch 22/50
25/25 [==============================] - 1s 21ms/step - loss: 0.0358 - accuracy: 0.9875
Epoch 23/50
25/25 [==============================] - 1s 25ms/step - loss: 0.0284 - accuracy: 0.9900
Epoch 24/50
25/25 [==============================] - 1s 37ms/step - loss: 0.0391 - accuracy: 0.9887
Epoch 25/50
25/25 [==============================] - 1s 37ms/step - loss: 0.0248 - accuracy: 0.9925
Epoch 26/50


25/25 [==============================] - 0s 19ms/step - loss: 0.0353 - accuracy: 0.9875
Epoch 25/50
25/25 [==============================] - 0s 12ms/step - loss: 0.0317 - accuracy: 0.9912
Epoch 26/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0343 - accuracy: 0.9925
Epoch 27/50
25/25 [==============================] - 1s 21ms/step - loss: 0.0259 - accuracy: 0.9937
Epoch 28/50
25/25 [==============================] - 0s 17ms/step - loss: 0.0382 - accuracy: 0.9900
Epoch 29/50
25/25 [==============================] - 0s 17ms/step - loss: 0.0267 - accuracy: 0.9900
Epoch 30/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0311 - accuracy: 0.9925
Epoch 31/50
25/25 [==============================] - 1s 27ms/step - loss: 0.0215 - accuracy: 0.9937
Epoch 32/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0308 - accuracy: 0.9912
Epoch 33/50
25/25 [==============================] - 1s 21ms/step - loss: 0.0279 - accuracy: 0.9925
Epoch 34/50


25/25 [==============================] - 1s 22ms/step - loss: 0.0321 - accuracy: 0.9925
Epoch 33/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0277 - accuracy: 0.9912
Epoch 34/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0502 - accuracy: 0.9862
Epoch 35/50
25/25 [==============================] - 0s 17ms/step - loss: 0.0380 - accuracy: 0.9862
Epoch 36/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0340 - accuracy: 0.9912
Epoch 37/50
25/25 [==============================] - 0s 18ms/step - loss: 0.0315 - accuracy: 0.9900
Epoch 38/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0233 - accuracy: 0.9937
Epoch 39/50
25/25 [==============================] - 1s 34ms/step - loss: 0.0338 - accuracy: 0.9912
Epoch 40/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0409 - accuracy: 0.9837
Epoch 41/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0334 - accuracy: 0.9887
Epoch 42/50


25/25 [==============================] - 1s 24ms/step - loss: 0.0732 - accuracy: 0.9712
Epoch 41/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0926 - accuracy: 0.9588
Epoch 42/50
25/25 [==============================] - 0s 11ms/step - loss: 0.0933 - accuracy: 0.9650
Epoch 43/50
25/25 [==============================] - 1s 29ms/step - loss: 0.0949 - accuracy: 0.9700
Epoch 44/50
25/25 [==============================] - 0s 18ms/step - loss: 0.0720 - accuracy: 0.9800
Epoch 45/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0736 - accuracy: 0.9750
Epoch 46/50
25/25 [==============================] - 0s 20ms/step - loss: 0.0817 - accuracy: 0.9762
Epoch 47/50
25/25 [==============================] - 1s 35ms/step - loss: 0.1023 - accuracy: 0.9712
Epoch 48/50
25/25 [==============================] - 1s 26ms/step - loss: 0.0686 - accuracy: 0.9737
Epoch 49/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0435 - accuracy: 0.9850
Epoch 50/50


25/25 [==============================] - 1s 24ms/step - loss: 0.0605 - accuracy: 0.9800
Epoch 49/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0503 - accuracy: 0.9812
Epoch 50/50
7/7 [==============================] - 0s 2ms/step - loss: 0.1288 - accuracy: 0.9700
------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/50
25/25 [==============================] - 1s 24ms/step - loss: 3.1767 - accuracy: 0.1963
Epoch 2/50
25/25 [==============================] - 1s 22ms/step - loss: 1.9350 - accuracy: 0.3587
Epoch 3/50
25/25 [==============================] - 1s 22ms/step - loss: 1.4301 - accuracy: 0.4975
Epoch 4/50
25/25 [==============================] - 1s 27ms/step - loss: 1.0968 - accuracy: 0.6075
Epoch 5/50
25/25 [==============================] - 1s 25ms/step - loss: 0.8864 - accuracy: 0.6875
Epoch 6/50
25/25 [==============================] - 1s 24ms/step - loss: 0.7482 - accuracy: 0.7362
Epoch 7/50
25/25 [======

In [13]:
print("Dictionary containing mean accuracies of each round of 5-fold cross-validation:")
print(accs)
print()

max_acc = max(i['mean'] for i in accs.values())
mean_accs = {key: max(val.values()) for key, val in accs.items()}
dropout_opt = max(mean_accs, key=mean_accs.get)
print("The highest validation accuracy of {:.2f}% is achieved with {}.".format(max_acc, dropout_opt))

Dictionary containing mean accuracies of each round of 5-fold cross-validation:
{'drop_out = 0.2': {'mean': 96.90000057220459, 'std': 1.3928403237842988}, 'drop_out = 0.3': {'mean': 97.5, 'std': 1.6431690074072547}, 'drop_out = 0.4': {'mean': 97.59999990463257, 'std': 1.5620512691839201}, 'drop_out = 0.5': {'mean': 97.2000002861023, 'std': 1.4352710493159182}}

The highest validation accuracy of 97.60% is achieved with drop_out = 0.4.


In [15]:
"""Performing 5-fold cross-validation on the number of filters."""

no_epochs = 50
no_filters = np.arange(40, 70, 5)
batch_size = int(batch_opt[-2:])
drop_out = float(dropout_opt[-2:])

accs = {}
losses = {}

for f in no_filters:
    loop(no_epochs=no_epochs, filters=f, batch_size=batch_size, drop_out=drop_out, 
         variable='{} filters'.format(f), no_folds=5)
                

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
25/25 [==============================] - 2s 46ms/step - loss: 2.6234 - accuracy: 0.2087
Epoch 2/50
25/25 [==============================] - 1s 38ms/step - loss: 1.6634 - accuracy: 0.4363
Epoch 3/50
25/25 [==============================] - 1s 32ms/step - loss: 1.1797 - accuracy: 0.5788
Epoch 4/50
25/25 [==============================] - 1s 33ms/step - loss: 0.9087 - accuracy: 0.6837
Epoch 5/50
25/25 [==============================] - 1s 50ms/step - loss: 0.6934 - accuracy: 0.7775
Epoch 6/50
25/25 [==============================] - 1s 29ms/step - loss: 0.5554 - accuracy: 0.8338
Epoch 7/50
25/25 [==============================] - 1s 31ms/step - loss: 0.4994 - accuracy: 0.8500
Epoch 8/50
25/25 [==============================] - 1s 28ms/step - loss: 0.4474 - accuracy: 0.8575
Epoch 9/50
25/25 [==============================] - 1s 31ms/step - loss: 0.3560 - accuracy: 0.8925
Epoch 10/50


25/25 [==============================] - 1s 25ms/step - loss: 0.3977 - accuracy: 0.8587
Epoch 9/50
25/25 [==============================] - 1s 30ms/step - loss: 0.3728 - accuracy: 0.8750
Epoch 10/50
25/25 [==============================] - 1s 33ms/step - loss: 0.3359 - accuracy: 0.8875
Epoch 11/50
25/25 [==============================] - 1s 50ms/step - loss: 0.2378 - accuracy: 0.9262
Epoch 12/50
25/25 [==============================] - 1s 30ms/step - loss: 0.2585 - accuracy: 0.9162
Epoch 13/50
25/25 [==============================] - 1s 36ms/step - loss: 0.1931 - accuracy: 0.9413
Epoch 14/50
25/25 [==============================] - 1s 45ms/step - loss: 0.2160 - accuracy: 0.9250
Epoch 15/50
25/25 [==============================] - 1s 36ms/step - loss: 0.1888 - accuracy: 0.9337
Epoch 16/50
25/25 [==============================] - 1s 42ms/step - loss: 0.1766 - accuracy: 0.9450
Epoch 17/50
25/25 [==============================] - 1s 41ms/step - loss: 0.1532 - accuracy: 0.9563
Epoch 18/50
2

25/25 [==============================] - 1s 27ms/step - loss: 0.1733 - accuracy: 0.9500
Epoch 17/50
25/25 [==============================] - 1s 39ms/step - loss: 0.1462 - accuracy: 0.9550
Epoch 18/50
25/25 [==============================] - 1s 45ms/step - loss: 0.1290 - accuracy: 0.9650
Epoch 19/50
25/25 [==============================] - 1s 33ms/step - loss: 0.1321 - accuracy: 0.9525
Epoch 20/50
25/25 [==============================] - 1s 34ms/step - loss: 0.0960 - accuracy: 0.9712
Epoch 21/50
25/25 [==============================] - 1s 30ms/step - loss: 0.1074 - accuracy: 0.9625
Epoch 22/50
25/25 [==============================] - 1s 25ms/step - loss: 0.0870 - accuracy: 0.9712
Epoch 23/50
25/25 [==============================] - 1s 40ms/step - loss: 0.0894 - accuracy: 0.9712
Epoch 24/50
25/25 [==============================] - 1s 40ms/step - loss: 0.1052 - accuracy: 0.9613
Epoch 25/50
25/25 [==============================] - 1s 38ms/step - loss: 0.1006 - accuracy: 0.9600
Epoch 26/50


25/25 [==============================] - 1s 38ms/step - loss: 0.0969 - accuracy: 0.9688
Epoch 25/50
25/25 [==============================] - 1s 35ms/step - loss: 0.0720 - accuracy: 0.9688
Epoch 26/50
25/25 [==============================] - 1s 38ms/step - loss: 0.0967 - accuracy: 0.9663
Epoch 27/50
25/25 [==============================] - 1s 37ms/step - loss: 0.0744 - accuracy: 0.9712
Epoch 28/50
25/25 [==============================] - 1s 49ms/step - loss: 0.0805 - accuracy: 0.9663
Epoch 29/50
25/25 [==============================] - 1s 33ms/step - loss: 0.0676 - accuracy: 0.9762
Epoch 30/50
25/25 [==============================] - 1s 59ms/step - loss: 0.0656 - accuracy: 0.9787
Epoch 31/50
25/25 [==============================] - 1s 55ms/step - loss: 0.0670 - accuracy: 0.9725
Epoch 32/50
25/25 [==============================] - 1s 47ms/step - loss: 0.0675 - accuracy: 0.9775
Epoch 33/50
25/25 [==============================] - 1s 46ms/step - loss: 0.0598 - accuracy: 0.9787
Epoch 34/50


25/25 [==============================] - 0s 20ms/step - loss: 0.0499 - accuracy: 0.9800
Epoch 33/50
25/25 [==============================] - 1s 23ms/step - loss: 0.0542 - accuracy: 0.9775
Epoch 34/50
25/25 [==============================] - 0s 16ms/step - loss: 0.0622 - accuracy: 0.9775
Epoch 35/50
25/25 [==============================] - 1s 20ms/step - loss: 0.0367 - accuracy: 0.9900
Epoch 36/50
25/25 [==============================] - 1s 36ms/step - loss: 0.0412 - accuracy: 0.9887
Epoch 37/50
25/25 [==============================] - 1s 29ms/step - loss: 0.0418 - accuracy: 0.9875
Epoch 38/50
25/25 [==============================] - 0s 14ms/step - loss: 0.0528 - accuracy: 0.9837
Epoch 39/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0388 - accuracy: 0.9912
Epoch 40/50
25/25 [==============================] - 0s 17ms/step - loss: 0.0483 - accuracy: 0.9825
Epoch 41/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0533 - accuracy: 0.9800
Epoch 42/50


25/25 [==============================] - 1s 27ms/step - loss: 0.0598 - accuracy: 0.9725
Epoch 41/50
25/25 [==============================] - 1s 22ms/step - loss: 0.0314 - accuracy: 0.9887
Epoch 42/50
25/25 [==============================] - 1s 22ms/step - loss: 0.0271 - accuracy: 0.9887
Epoch 43/50
25/25 [==============================] - 1s 22ms/step - loss: 0.0409 - accuracy: 0.9862
Epoch 44/50
25/25 [==============================] - 1s 22ms/step - loss: 0.0382 - accuracy: 0.9862
Epoch 45/50
25/25 [==============================] - 0s 20ms/step - loss: 0.0339 - accuracy: 0.9862
Epoch 46/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0220 - accuracy: 0.9925
Epoch 47/50
25/25 [==============================] - 1s 22ms/step - loss: 0.0235 - accuracy: 0.9925
Epoch 48/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0410 - accuracy: 0.9912
Epoch 49/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0177 - accuracy: 0.9975
Epoch 50/50


25/25 [==============================] - 0s 16ms/step - loss: 0.0221 - accuracy: 0.9962
Epoch 49/50
25/25 [==============================] - 1s 23ms/step - loss: 0.0290 - accuracy: 0.9912
Epoch 50/50
7/7 [==============================] - 0s 2ms/step - loss: 0.0753 - accuracy: 0.9800
------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/50
25/25 [==============================] - 1s 13ms/step - loss: 2.2915 - accuracy: 0.2937
Epoch 2/50
25/25 [==============================] - 1s 21ms/step - loss: 1.1862 - accuracy: 0.5813
Epoch 3/50
25/25 [==============================] - 1s 30ms/step - loss: 0.8161 - accuracy: 0.7337
Epoch 4/50
25/25 [==============================] - 0s 15ms/step - loss: 0.6054 - accuracy: 0.7950
Epoch 5/50
25/25 [==============================] - 1s 20ms/step - loss: 0.4769 - accuracy: 0.8500
Epoch 6/50
25/25 [==============================] - 1s 27ms/step - loss: 0.3582 - accuracy: 0.8900
Epoch 7/50
25/25 [======

25/25 [==============================] - 1s 19ms/step - loss: 0.5366 - accuracy: 0.8213
Epoch 5/50
25/25 [==============================] - 0s 18ms/step - loss: 0.4485 - accuracy: 0.8450
Epoch 6/50
25/25 [==============================] - 1s 19ms/step - loss: 0.3199 - accuracy: 0.9025
Epoch 7/50
25/25 [==============================] - 0s 16ms/step - loss: 0.3021 - accuracy: 0.9112
Epoch 8/50
25/25 [==============================] - 0s 20ms/step - loss: 0.2576 - accuracy: 0.9262
Epoch 9/50
25/25 [==============================] - 1s 23ms/step - loss: 0.2127 - accuracy: 0.9325
Epoch 10/50
25/25 [==============================] - 0s 13ms/step - loss: 0.1808 - accuracy: 0.9375
Epoch 11/50
25/25 [==============================] - 0s 18ms/step - loss: 0.1637 - accuracy: 0.9513
Epoch 12/50
25/25 [==============================] - 0s 17ms/step - loss: 0.1664 - accuracy: 0.9475
Epoch 13/50
25/25 [==============================] - 1s 20ms/step - loss: 0.1515 - accuracy: 0.9525
Epoch 14/50
25/25

25/25 [==============================] - 1s 44ms/step - loss: 0.0907 - accuracy: 0.9737
Epoch 13/50
25/25 [==============================] - 1s 47ms/step - loss: 0.1009 - accuracy: 0.9663
Epoch 14/50
25/25 [==============================] - 1s 37ms/step - loss: 0.1003 - accuracy: 0.9700
Epoch 15/50
25/25 [==============================] - 1s 37ms/step - loss: 0.0981 - accuracy: 0.9712
Epoch 16/50
25/25 [==============================] - 1s 41ms/step - loss: 0.0868 - accuracy: 0.9700
Epoch 17/50
25/25 [==============================] - 1s 48ms/step - loss: 0.0836 - accuracy: 0.9775
Epoch 18/50
25/25 [==============================] - 1s 33ms/step - loss: 0.0657 - accuracy: 0.9825
Epoch 19/50
25/25 [==============================] - 1s 48ms/step - loss: 0.0781 - accuracy: 0.9737
Epoch 20/50
25/25 [==============================] - 1s 41ms/step - loss: 0.0606 - accuracy: 0.9862
Epoch 21/50
25/25 [==============================] - 1s 45ms/step - loss: 0.0424 - accuracy: 0.9862
Epoch 22/50


25/25 [==============================] - 1s 35ms/step - loss: 0.0774 - accuracy: 0.9825
Epoch 21/50
25/25 [==============================] - 1s 44ms/step - loss: 0.0682 - accuracy: 0.9812
Epoch 22/50
25/25 [==============================] - 1s 40ms/step - loss: 0.0588 - accuracy: 0.9825
Epoch 23/50
25/25 [==============================] - 1s 34ms/step - loss: 0.0553 - accuracy: 0.9850
Epoch 24/50
25/25 [==============================] - 1s 47ms/step - loss: 0.0528 - accuracy: 0.9812
Epoch 25/50
25/25 [==============================] - 1s 55ms/step - loss: 0.0519 - accuracy: 0.9862
Epoch 26/50
25/25 [==============================] - 1s 30ms/step - loss: 0.0531 - accuracy: 0.9812
Epoch 27/50
25/25 [==============================] - 1s 44ms/step - loss: 0.0477 - accuracy: 0.9825
Epoch 28/50
25/25 [==============================] - 1s 51ms/step - loss: 0.0478 - accuracy: 0.9862
Epoch 29/50
25/25 [==============================] - 1s 33ms/step - loss: 0.0336 - accuracy: 0.9912
Epoch 30/50


In [16]:
print("Dictionary containing mean accuracies of each round of 5-fold cross-validation:")
print(accs)
print()

max_acc = max(i['mean'] for i in accs.values())
mean_accs = {key: max(val.values()) for key, val in accs.items()}
filter_opt = max(mean_accs, key=mean_accs.get)
print("The highest validation accuracy of {:.2f}% is achieved with {}.".format(max_acc, filter_opt))

Dictionary containing mean accuracies of each round of 5-fold cross-validation:
{'40 filters': {'mean': 97.10000038146973, 'std': 0.7348489798269677}, '45 filters': {'mean': 97.00000166893005, 'std': 1.4491383491696914}, '50 filters': {'mean': 96.90000057220459, 'std': 0.6633266617693734}, '55 filters': {'mean': 97.50000238418579, 'std': 0.44721316900383795}, '60 filters': {'mean': 97.30000019073486, 'std': 1.077034258633658}, '65 filters': {'mean': 97.8000009059906, 'std': 0.9273626362568285}}

The highest validation accuracy of 97.80% is achieved with 65 filters.


In [17]:
"""Performing 5-fold cross-validation on the number of filters."""

no_epochs = np.arange(30, 65, 5)
no_filters = int(filter_opt[0:2])
batch_size = int(batch_opt[-2:])
drop_out = float(dropout_opt[-2:])

accs = {}
losses = {}

for n in no_epochs:
    loop(no_epochs=n, filters=no_filters, batch_size=batch_size, drop_out=drop_out, 
         variable='{} epochs'.format(n), no_folds=5)
                

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/30
25/25 [==============================] - 1s 29ms/step - loss: 2.2491 - accuracy: 0.3088
Epoch 2/30
25/25 [==============================] - 1s 40ms/step - loss: 1.0644 - accuracy: 0.6075
Epoch 3/30
25/25 [==============================] - 1s 27ms/step - loss: 0.7043 - accuracy: 0.7550
Epoch 4/30
25/25 [==============================] - 1s 56ms/step - loss: 0.5449 - accuracy: 0.8150
Epoch 5/30
25/25 [==============================] - 2s 61ms/step - loss: 0.3889 - accuracy: 0.8637
Epoch 6/30
25/25 [==============================] - 1s 40ms/step - loss: 0.2679 - accuracy: 0.9162
Epoch 7/30
25/25 [==============================] - 1s 50ms/step - loss: 0.2443 - accuracy: 0.9137
Epoch 8/30
25/25 [==============================] - 1s 35ms/step - loss: 0.1869 - accuracy: 0.9350
Epoch 9/30
25/25 [==============================] - 1s 28ms/step - loss: 0.1696 - accuracy: 0.9475
Epoch 10/30


25/25 [==============================] - 1s 50ms/step - loss: 0.5467 - accuracy: 0.8250
Epoch 5/35
25/25 [==============================] - 1s 55ms/step - loss: 0.4098 - accuracy: 0.8525
Epoch 6/35
25/25 [==============================] - 2s 69ms/step - loss: 0.2894 - accuracy: 0.9100
Epoch 7/35
25/25 [==============================] - 1s 43ms/step - loss: 0.2547 - accuracy: 0.9250
Epoch 8/35
25/25 [==============================] - 1s 51ms/step - loss: 0.2461 - accuracy: 0.9200
Epoch 9/35
25/25 [==============================] - 1s 38ms/step - loss: 0.1661 - accuracy: 0.9500
Epoch 10/35
25/25 [==============================] - 1s 40ms/step - loss: 0.1796 - accuracy: 0.9425
Epoch 11/35
25/25 [==============================] - 1s 36ms/step - loss: 0.1486 - accuracy: 0.9475
Epoch 12/35
25/25 [==============================] - 1s 33ms/step - loss: 0.1169 - accuracy: 0.9588
Epoch 13/35
25/25 [==============================] - 1s 38ms/step - loss: 0.1024 - accuracy: 0.9650
Epoch 14/35
25/25

25/25 [==============================] - 2s 65ms/step - loss: 0.0719 - accuracy: 0.9737
Epoch 21/35
25/25 [==============================] - 1s 43ms/step - loss: 0.0773 - accuracy: 0.9675
Epoch 22/35
25/25 [==============================] - 1s 54ms/step - loss: 0.0592 - accuracy: 0.9812
Epoch 23/35
25/25 [==============================] - 1s 47ms/step - loss: 0.0432 - accuracy: 0.9887
Epoch 24/35
25/25 [==============================] - 1s 47ms/step - loss: 0.0613 - accuracy: 0.9787
Epoch 25/35
25/25 [==============================] - 1s 49ms/step - loss: 0.0435 - accuracy: 0.9862
Epoch 26/35
25/25 [==============================] - 1s 56ms/step - loss: 0.0536 - accuracy: 0.9862
Epoch 27/35
25/25 [==============================] - 1s 44ms/step - loss: 0.0444 - accuracy: 0.9850
Epoch 28/35
25/25 [==============================] - 1s 49ms/step - loss: 0.0589 - accuracy: 0.9800
Epoch 29/35
25/25 [==============================] - 1s 41ms/step - loss: 0.0469 - accuracy: 0.9825
Epoch 30/35


25/25 [==============================] - 1s 41ms/step - loss: 0.0528 - accuracy: 0.9812
Epoch 22/40
25/25 [==============================] - 1s 41ms/step - loss: 0.0500 - accuracy: 0.9800
Epoch 23/40
25/25 [==============================] - 1s 53ms/step - loss: 0.0560 - accuracy: 0.9812
Epoch 24/40
25/25 [==============================] - 1s 45ms/step - loss: 0.0467 - accuracy: 0.9825
Epoch 25/40
25/25 [==============================] - 1s 44ms/step - loss: 0.0504 - accuracy: 0.9825
Epoch 26/40
25/25 [==============================] - 1s 24ms/step - loss: 0.0478 - accuracy: 0.9862
Epoch 27/40
25/25 [==============================] - 1s 40ms/step - loss: 0.0488 - accuracy: 0.9825
Epoch 28/40
25/25 [==============================] - 1s 38ms/step - loss: 0.0512 - accuracy: 0.9812
Epoch 29/40
25/25 [==============================] - 1s 31ms/step - loss: 0.0355 - accuracy: 0.9900
Epoch 30/40
25/25 [==============================] - 1s 46ms/step - loss: 0.0347 - accuracy: 0.9912
Epoch 31/40


25/25 [==============================] - 1s 54ms/step - loss: 0.2341 - accuracy: 0.9325
Epoch 8/45
25/25 [==============================] - 1s 54ms/step - loss: 0.2003 - accuracy: 0.9337
Epoch 9/45
25/25 [==============================] - 1s 38ms/step - loss: 0.1687 - accuracy: 0.9475
Epoch 10/45
25/25 [==============================] - 1s 44ms/step - loss: 0.1386 - accuracy: 0.9550
Epoch 11/45
25/25 [==============================] - 0s 17ms/step - loss: 0.1343 - accuracy: 0.9475
Epoch 12/45
25/25 [==============================] - 1s 29ms/step - loss: 0.1319 - accuracy: 0.9563
Epoch 13/45
25/25 [==============================] - 1s 48ms/step - loss: 0.0893 - accuracy: 0.9712
Epoch 14/45
25/25 [==============================] - 1s 26ms/step - loss: 0.1096 - accuracy: 0.9638
Epoch 15/45
25/25 [==============================] - 1s 45ms/step - loss: 0.0733 - accuracy: 0.9762
Epoch 16/45
25/25 [==============================] - 1s 38ms/step - loss: 0.0761 - accuracy: 0.9725
Epoch 17/45
25

...

25/25 [==============================] - 1s 41ms/step - loss: 0.0214 - accuracy: 0.9925
Epoch 39/50
25/25 [==============================] - 1s 42ms/step - loss: 0.0232 - accuracy: 0.9950
Epoch 40/50
25/25 [==============================] - 1s 24ms/step - loss: 0.0327 - accuracy: 0.9837
Epoch 41/50
25/25 [==============================] - 1s 29ms/step - loss: 0.0207 - accuracy: 0.9925
Epoch 42/50
25/25 [==============================] - 1s 35ms/step - loss: 0.0292 - accuracy: 0.9900
Epoch 43/50
25/25 [==============================] - 1s 52ms/step - loss: 0.0241 - accuracy: 0.9900
Epoch 44/50
25/25 [==============================] - 1s 27ms/step - loss: 0.0182 - accuracy: 0.9950
Epoch 45/50
25/25 [==============================] - 0s 19ms/step - loss: 0.0176 - accuracy: 0.9975
Epoch 46/50
25/25 [==============================] - 1s 35ms/step - loss: 0.0354 - accuracy: 0.9887
Epoch 47/50
25/25 [==============================] - 1s 41ms/step - loss: 0.0311 - accuracy: 0.9900
Epoch 48/50


25/25 [==============================] - 1s 33ms/step - loss: 0.0188 - accuracy: 0.9937
Epoch 42/55
25/25 [==============================] - 1s 40ms/step - loss: 0.0193 - accuracy: 0.9950
Epoch 43/55
25/25 [==============================] - 1s 31ms/step - loss: 0.0170 - accuracy: 0.9950
Epoch 44/55
25/25 [==============================] - 1s 54ms/step - loss: 0.0167 - accuracy: 0.9937
Epoch 45/55
25/25 [==============================] - 1s 47ms/step - loss: 0.0146 - accuracy: 0.9962
Epoch 46/55
25/25 [==============================] - 1s 47ms/step - loss: 0.0124 - accuracy: 0.9950
Epoch 47/55
25/25 [==============================] - 1s 29ms/step - loss: 0.0184 - accuracy: 0.9937
Epoch 48/55
25/25 [==============================] - 1s 30ms/step - loss: 0.0246 - accuracy: 0.9912
Epoch 49/55
25/25 [==============================] - 1s 46ms/step - loss: 0.0148 - accuracy: 0.9950
Epoch 50/55
25/25 [==============================] - 1s 36ms/step - loss: 0.0264 - accuracy: 0.9912
Epoch 51/55


25/25 [==============================] - 2s 72ms/step - loss: 0.0171 - accuracy: 0.9937
Epoch 35/55
25/25 [==============================] - 1s 38ms/step - loss: 0.0127 - accuracy: 0.9987
Epoch 36/55
25/25 [==============================] - 1s 36ms/step - loss: 0.0250 - accuracy: 0.9912
Epoch 37/55
25/25 [==============================] - 1s 46ms/step - loss: 0.0328 - accuracy: 0.9912
Epoch 38/55
25/25 [==============================] - 1s 43ms/step - loss: 0.0265 - accuracy: 0.9925
Epoch 39/55
25/25 [==============================] - 1s 58ms/step - loss: 0.0205 - accuracy: 0.9962
Epoch 40/55
25/25 [==============================] - 1s 24ms/step - loss: 0.0192 - accuracy: 0.9937
Epoch 41/55
25/25 [==============================] - 1s 29ms/step - loss: 0.0154 - accuracy: 0.9937
Epoch 42/55
25/25 [==============================] - 1s 45ms/step - loss: 0.0097 - accuracy: 0.9987
Epoch 43/55
25/25 [==============================] - 1s 34ms/step - loss: 0.0255 - accuracy: 0.9937
Epoch 44/55


25/25 [==============================] - 1s 46ms/step - loss: 0.0916 - accuracy: 0.9762
Epoch 18/60
25/25 [==============================] - 1s 39ms/step - loss: 0.0444 - accuracy: 0.9862
Epoch 19/60
25/25 [==============================] - 1s 40ms/step - loss: 0.0559 - accuracy: 0.9800
Epoch 20/60
25/25 [==============================] - 1s 44ms/step - loss: 0.0756 - accuracy: 0.9812
Epoch 21/60
25/25 [==============================] - 2s 66ms/step - loss: 0.0448 - accuracy: 0.9900
Epoch 22/60
25/25 [==============================] - 1s 47ms/step - loss: 0.0576 - accuracy: 0.9800
Epoch 23/60
25/25 [==============================] - 1s 26ms/step - loss: 0.0694 - accuracy: 0.9762
Epoch 24/60
25/25 [==============================] - 1s 40ms/step - loss: 0.0421 - accuracy: 0.9875
Epoch 25/60
25/25 [==============================] - 1s 53ms/step - loss: 0.0453 - accuracy: 0.9900
Epoch 26/60
25/25 [==============================] - 1s 31ms/step - loss: 0.0387 - accuracy: 0.9875
Epoch 27/60


25/25 [==============================] - 2s 60ms/step - loss: 0.0214 - accuracy: 0.9887
Epoch 58/60
25/25 [==============================] - 1s 38ms/step - loss: 0.0088 - accuracy: 0.9975
Epoch 59/60
25/25 [==============================] - 1s 36ms/step - loss: 0.0144 - accuracy: 0.9950
Epoch 60/60
7/7 [==============================] - 0s 2ms/step - loss: 0.1681 - accuracy: 0.9650


In [18]:
print("Dictionary containing mean accuracies of each round of 5-fold cross-validation:")
print(accs)
print()

max_acc = max(i['mean'] for i in accs.values())
mean_accs = {key: max(val.values()) for key, val in accs.items()}
epoch_opt = max(mean_accs, key=mean_accs.get)
print("The highest validation accuracy of {:.2f}% is achieved with {}.".format(max_acc, epoch_opt))   # 98.0% for 35 epochs

Dictionary containing mean accuracies of each round of 5-fold cross-validation:
{'30 epochs': {'mean': 97.10000157356262, 'std': 0.6633263023388678}, '35 epochs': {'mean': 97.40000128746033, 'std': 0.8602336464154917}, '40 epochs': {'mean': 97.70000100135803, 'std': 0.8124048265344118}, '45 epochs': {'mean': 98.00000190734863, 'std': 0.5477220351562305}, '50 epochs': {'mean': 97.10000038146973, 'std': 1.3190917778983329}, '55 epochs': {'mean': 97.20000147819519, 'std': 0.509903101602391}, '60 epochs': {'mean': 97.00000047683716, 'std': 1.0000014305128957}}

The highest validation accuracy of 98.00% is achieved with 45 epochs.


In [23]:
# Compile the model
model = CNN(filters=int(filter_opt[0:2]), kernel_size=3, drop_out=float(dropout_opt[-2:]))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fitting the model to the training data.
model.fit(x_train, y_train, batch_size=int(batch_opt[-2:]), epochs=int(epoch_opt[0:2])) 

# Evaluating the model performance on the test dataset.
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=int(batch_opt[-2:]))
print('\nTest accuracy: {:.2f}%'.format(test_acc*100))

Epoch 1/45
32/32 [==============================] - 2s 43ms/step - loss: 2.1253 - accuracy: 0.3480
Epoch 2/45
32/32 [==============================] - 1s 48ms/step - loss: 0.8882 - accuracy: 0.7070
Epoch 3/45
32/32 [==============================] - 1s 33ms/step - loss: 0.5715 - accuracy: 0.8180
Epoch 4/45
32/32 [==============================] - 2s 63ms/step - loss: 0.3770 - accuracy: 0.8780
Epoch 5/45
32/32 [==============================] - 1s 41ms/step - loss: 0.2783 - accuracy: 0.8970
Epoch 6/45
32/32 [==============================] - 2s 54ms/step - loss: 0.2356 - accuracy: 0.9180
Epoch 7/45
32/32 [==============================] - 1s 32ms/step - loss: 0.1843 - accuracy: 0.9430
Epoch 8/45
32/32 [==============================] - 1s 24ms/step - loss: 0.1864 - accuracy: 0.9340
Epoch 9/45
32/32 [==============================] - 1s 25ms/step - loss: 0.1574 - accuracy: 0.9530
Epoch 10/45
32/32 [==============================] - 2s 54ms/step - loss: 0.1315 - accuracy: 0.9520
Epoch 11/

In [27]:
model.save('saved_model/our_cnn')

INFO:tensorflow:Assets written to: saved_model/our_cnn\assets


INFO:tensorflow:Assets written to: saved_model/our_cnn\assets
